In [1]:

import numpy as np
from tqdm import tqdm
import cv2 as cv
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense,Dropout,Flatten,Conv2D
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential

In [2]:
import tensorflow as tf

In [3]:
import gdown


In [4]:
url="https://drive.google.com/uc?id=1BJr82HLPoynQW-UPB8AksmlwtPhmIN6O"

In [5]:
url="https://drive.google.com/uc?id=1rsf2qA78kZ34OSIrDJhbmK5S_mKJkt_x"

In [6]:
output = 'data.zip'
gdown.download(url, output, quiet=False)
gdown.extractall('data.zip')

Downloading...
From: https://drive.google.com/uc?id=1rsf2qA78kZ34OSIrDJhbmK5S_mKJkt_x
To: /content/data.zip
100%|██████████| 126M/126M [00:02<00:00, 54.4MB/s]


['images/images/train/angry/0.jpg',
 'images/images/train/angry/1.jpg',
 'images/images/train/angry/10.jpg',
 'images/images/train/angry/10002.jpg',
 'images/images/train/angry/10016.jpg',
 'images/images/train/angry/10037.jpg',
 'images/images/train/angry/10038.jpg',
 'images/images/train/angry/10057.jpg',
 'images/images/train/angry/10063.jpg',
 'images/images/train/angry/10069.jpg',
 'images/images/train/angry/10100.jpg',
 'images/images/train/angry/10105.jpg',
 'images/images/train/angry/10115.jpg',
 'images/images/train/angry/10122.jpg',
 'images/images/train/angry/10142.jpg',
 'images/images/train/angry/1015.jpg',
 'images/images/train/angry/10154.jpg',
 'images/images/train/angry/10155.jpg',
 'images/images/train/angry/10158.jpg',
 'images/images/train/angry/1016.jpg',
 'images/images/train/angry/10167.jpg',
 'images/images/train/angry/10178.jpg',
 'images/images/train/angry/1018.jpg',
 'images/images/train/angry/10181.jpg',
 'images/images/train/angry/10192.jpg',
 'images/image

In [7]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [8]:
IMG_SIZE=(128,128)

In [9]:
def load_data(dir_path,IMG_SIZE):
    X=[]
    Y=[]
    i=0
    labels=dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i]=path
            for file in os.listdir(dir_path+'/'+path):
                if not file.startswith('.'):
                    img=cv.imread(dir_path+'/'+path+'/'+file)
                    img=img.astype('float32')/255
                    resized=cv.resize(img,IMG_SIZE,interpolation = cv.INTER_AREA)
                    X.append(resized)
                    Y.append(i)
            i+=1
    
    X=np.array(X)
    Y=np.array(Y)
    print(f'{len(X)} images loaded from {dir_path}')
    return X,Y,labels
                    

In [10]:
TRAIN_DIR="images/train"

In [11]:
X_train,Y_train,train_labels=load_data(TRAIN_DIR,IMG_SIZE)

100%|██████████| 7/7 [00:05<00:00,  1.25it/s]


28821 images loaded from images/train


In [12]:
train_labels

{0: 'angry',
 1: 'disgust',
 2: 'fear',
 3: 'happy',
 4: 'neutral',
 5: 'sad',
 6: 'surprise'}

In [13]:
from keras.utils.np_utils import to_categorical


In [14]:
Y_train=to_categorical(Y_train,num_classes=7)
Y_train.shape

(28821, 7)

In [15]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD

In [16]:

EPOCHS = 50
BS = 128
DROPOUT_RATE = 0.3
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

In [17]:
model7 = Sequential()
model7.add(BatchNormalization(input_shape=(128,128,3)))
model7.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(128,128,3),name="conv1"))
model7.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool1"))
model7.add(Dropout(0.2))
model7.add(Conv2D(32, (3, 3), activation='relu',padding='same',name="conv2"))
model7.add(BatchNormalization())
model7.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))         
model7.add(Dropout(0.2))
model7.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv3"))
model7.add(BatchNormalization())
model7.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model7.add(Dropout(0.2))
model7.add(Conv2D(64, (3, 3), activation='relu',padding='same',name="conv4"))
model7.add(BatchNormalization())
model7.add(MaxPooling2D(pool_size=(2, 2),name="maxpool4"))
model7.add(Dropout(0.2))
model7.add(Flatten())
model7.add(Dense(1024, activation='relu',name='fc1'))
model7.add(Dropout(DROPOUT_RATE))
model7.add(BatchNormalization())
model7.add(Dense(7, activation='softmax',name='fcsoftmax'))

In [18]:
#TODO: weight decay of 0.0001...initial learning rate is set to 0.01 and reduced by a factor of 2 at every 25 epoch
sgd = SGD(learning_rate=SGD_LEARNING_RATE,momentum=0.9, decay=SGD_DECAY, nesterov=True)
model7.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [19]:
history=model7.fit(X_train,Y_train,epochs=100,shuffle=True)

Epoch 1/100
901/901 [==============================] - 59s 62ms/step - loss: 2.2330 - accuracy: 0.2817
Epoch 2/100
901/901 [==============================] - 55s 61ms/step - loss: 1.9469 - accuracy: 0.3415
Epoch 3/100
901/901 [==============================] - 55s 61ms/step - loss: 1.6851 - accuracy: 0.4048
Epoch 4/100
901/901 [==============================] - 55s 61ms/step - loss: 1.5028 - accuracy: 0.4524
Epoch 5/100
901/901 [==============================] - 55s 61ms/step - loss: 1.3737 - accuracy: 0.4929
Epoch 6/100
901/901 [==============================] - 55s 61ms/step - loss: 1.2674 - accuracy: 0.5278
Epoch 7/100
901/901 [==============================] - 55s 61ms/step - loss: 1.1699 - accuracy: 0.5628
Epoch 8/100
901/901 [==============================] - 55s 62ms/step - loss: 1.0853 - accuracy: 0.5898
Epoch 9/100
901/901 [==============================] - 55s 61ms/step - loss: 1.0006 - accuracy: 0.6306
Epoch 10/100
901/901 [==============================] - 55s 61ms/step - l

KeyboardInterrupt: ignored

In [20]:
model7.save('Face_Model.h5')